In [ ]:
%%capture
import datajoint as dj
dj.config['database.user'] = 'stelios'
dj.config['database.password'] = 'stelios#1'

from caveclient import CAVEclient
client = CAVEclient()
client.auth.save_token(token='27dde2c10c1ce7a6d1169fc594addc12', overwrite=True)

from microns_coregistration_api.dashboard.cell_type_trainer import CellTypeTrainer
from microns_utils.widget_utils import DataJointConnect


In [ ]:
%%javascript
require.undef('user_widget');

    define('user_widget', ["@jupyter-widgets/base"], function (widgets) {

        
        var UserView = widgets.DOMWidgetView.extend({
            initialize: function(attributes, options) {

            this.response = fetch(
                    '/hub/dashboards-api/hub-info/user',
                    { 
                    mode: 'no-cors', 
                    credentials: 'same-origin',
                    headers: new Headers({'Access-Control-Allow-Origin':'*'}) 
                });
                this.response.then( response => {
                    this.result = response.json();
                    this.result.then( json => {
                        this.model.set('value', json);
                        this.model.set('name', json.name);
                        this.model.save_changes();
                    });

                });
            
            },
            
            render: async function () {
                await this.response;
                await this.result;
                var json = this.model.get('value');
                var text = 'No user';
                if (json.hasOwnProperty('name')) {
                    text = 'Rendered by Javascript: '+json['name'];
                }
                this.el.appendChild(document.createTextNode(text));

            },
        
        });

        return {
            UserView: UserView
        };
    });

In [ ]:
from traitlets import Unicode, Dict, Unicode
from ipywidgets import DOMWidget, register
from ipywidgets import link

@register
class DashboardUser(DOMWidget):
    """
    Get JupyterHub user info
    https://gist.github.com/danlester/ac1d5f29358ce1950482f8e7d4301f86
    """
    _view_name = Unicode('UserView').tag(sync=True)
    _view_module = Unicode('user_widget').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)

    value = Dict({}, help="User info").tag(sync=True)
    name = Unicode('').tag(sync=True)

In [ ]:
d = DataJointConnect(disable_after_submitting=True, action_on_submit=CellTypeTrainer, kwargs_for_action_on_submit=dict(display=True, user_info=user.value))

: 